In [2]:
# 1、机器学习的DictVectorizer练习，文本分词，CountVectorizer, TfidfVectorizer练习
# 2、归一化，标准化，缺失值插补练习
# 3、练习特征预处理中的VarianceThreshold和PCA
# 4、完成对数据集的data，target，DESCR，feature_names，target_names的理解

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
import jieba
import numpy as np
from sklearn.impute import SimpleImputer

d:\py\Lib\site-packages\jieba\_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


# 1 Dictionary 中含有字符串的时候（当成类别），如何做特征抽取

In [4]:

def dictvec():
    """
    字典数据抽取
    :return: None
    """
    # 初始化DictVectorizer，用于数据特征抽取
    # sparse = True,输出的是每个不为零位置的坐标(稀疏矩阵) , 不推荐
    dv1 = DictVectorizer(sparse=True)   
    # sparse = False,输出的是密集矩阵, 推荐，默认
    dv2 = DictVectorizer(sparse=False)  

    # 三个样本，每个样本都是一个字典
    dict = [{'city': '北京', 'temperature': 100},
            {'city': '上海', 'temperature': 60},
            {'city': '深圳', 'temperature': 30}]

    data1 = dv1.fit_transform( dict )
    data2 = dv2.fit_transform( dict )
    print('Sparse=True：\n',data1)
    print('-' * 50)
    print('Sparse=False：\n',data2 )
    print('-' * 50)
    # 字典中的一些类别数据，分别进行转换成特征
    print('特征名：')
    print(dv1.get_feature_names_out()) #把每个特征名打印出来
    print(dv2.get_feature_names_out()) #把每个特征名打印出来
    print('-' * 50)
    print(dv1.inverse_transform(data1))  #去看每个特征代表的含义，逆转回去,这里不重要
    print(dv2.inverse_transform(data2))  #去看每个特征代表的含义，逆转回去,这里不重要
    return None

dictvec()

Sparse=True：
 <Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6 stored elements and shape (3, 4)>
  Coords	Values
  (0, 1)	1.0
  (0, 3)	100.0
  (1, 0)	1.0
  (1, 3)	60.0
  (2, 2)	1.0
  (2, 3)	30.0
--------------------------------------------------
Sparse=False：
 [[  0.   1.   0. 100.]
 [  1.   0.   0.  60.]
 [  0.   0.   1.  30.]]
--------------------------------------------------
特征名：
['city=上海' 'city=北京' 'city=深圳' 'temperature']
['city=上海' 'city=北京' 'city=深圳' 'temperature']
--------------------------------------------------
[{'city=北京': 1.0, 'temperature': 100.0}, {'city=上海': 1.0, 'temperature': 60.0}, {'city=深圳': 1.0, 'temperature': 30.0}]
[{'city=北京': 1.0, 'temperature': 100.0}, {'city=上海': 1.0, 'temperature': 60.0}, {'city=深圳': 1.0, 'temperature': 30.0}]


# 2 一段英文文本如何变为数值类型

In [5]:

def couvec():
    # 实例化CountVectorizer
    # max_df, min_df整数：min：至少出现几次的词才被统计。max：最多出现几次的词才被统计
    # max_df, min_df小数(0-1之间的）：按在整个文本中最多/最少出现的百分比进行保留
    # 默认会去除单个字母的单词，默认认为这个词对整个样本没有影响,认为其没有语义

    vector = CountVectorizer(min_df=2)  # 至少出现2次的词才被统计

    # 调用fit_transform输入并转换数据

    res = vector.fit_transform(
        ["life is  short,i like python life",
         "life is too long,i dislike python",
         "life is short"])

    # 打印结果,把每个词都分离了
    print('特征词：\n',vector.get_feature_names_out())
    print('-' * 50)

    print('分词结果保存在稀疏矩阵内：\n',res)
    print('-' * 50)

    print(type(res))
    # 对照feature_names，标记每个词出现的次数
    print('-' * 50)

    #稀疏矩阵转换为数组
    print('稀疏矩阵转换为数组:\n',res.toarray())  
    print('-' * 50)

    #拿每个样本里的特征进行显示
    print(vector.inverse_transform(res)) #不重要


couvec()

特征词：
 ['is' 'life' 'python' 'short']
--------------------------------------------------
分词结果保存在稀疏矩阵内：
 <Compressed Sparse Row sparse matrix of dtype 'int64'
	with 10 stored elements and shape (3, 4)>
  Coords	Values
  (0, 1)	2
  (0, 0)	1
  (0, 3)	1
  (0, 2)	1
  (1, 1)	1
  (1, 0)	1
  (1, 2)	1
  (2, 1)	1
  (2, 0)	1
  (2, 3)	1
--------------------------------------------------
<class 'scipy.sparse._csr.csr_matrix'>
--------------------------------------------------
稀疏矩阵转换为数组:
 [[1 2 1 1]
 [1 1 1 0]
 [1 1 0 1]]
--------------------------------------------------
[array(['life', 'is', 'short', 'python'], dtype='<U6'), array(['life', 'is', 'python'], dtype='<U6'), array(['life', 'is', 'short'], dtype='<U6')]


# 一段汉字文本如何数值化，对于汉字不能用空格来分割

In [6]:

def countvec():
    """
    使用CountVectorizer对文本进行特征值化 
    $注意 ： 和单个字母一样，单个汉字单个字母不统计，因为单个汉字字母没有意义   
    """
    cv = CountVectorizer()

    data = cv.fit_transform(["人生苦短，我喜欢 python python", "人生漫长，不用 python"])

    print('特征词： ',cv.get_feature_names_out())
    print('-' * 50)

    #默认会变成稀疏矩阵存储，只记录非零位置
    print(data)  
    print('-' * 50)

    print(data.toarray())
    return None


countvec()

特征词：  ['python' '不用' '人生漫长' '人生苦短' '我喜欢']
--------------------------------------------------
<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 6 stored elements and shape (2, 5)>
  Coords	Values
  (0, 3)	1
  (0, 4)	1
  (0, 0)	2
  (1, 0)	1
  (1, 2)	1
  (1, 1)	1
--------------------------------------------------
[[2 0 0 1 1]
 [1 1 1 0 0]]


## 1.3 掌握如何对中文进行分词(jieba)

In [7]:

def cutword():
    """
    通过 jieba.cut 对中文进行分词
    """
    con1 = jieba.cut("今天很残酷，明天更残酷，后天很美好，但绝对大部分是死在明天晚上，所以每个人不要放弃今天。")

    con2 = jieba.cut("我们看到的从很远星系来的光是在几百万年之前发出的，这样当我们看到宇宙时，我们是在看它的过去。")

    con3 = jieba.cut(
        "如果只用一种方式了解某样事物，你就不会真正了解它。了解事物真正含义的秘密取决于如何将其与我们所了解的事物相联系。")

    # 转换成列表
    print('jieba分词器产生的结果类型是：',type(con1))
    print('-' * 50)

    # 把生成器转换成列表
    content1 = list(con1)
    content2 = list(con2)
    content3 = list(con3)
    print(content1)
    print(content2)
    print(content3)

    # 把列表转换成字符串,每个词之间用空格隔开
    print('-' * 50)
    c1 = ' '.join(content1)
    c2 = ' '.join(content2)
    c3 = ' '.join(content3)
    return c1, c2, c3


def chinese_vec():
    """
    把jieba分好的词,对中文进行特征值化
    """
    c1, c2, c3 = cutword()  #jieba分词好的中文文本
    print('-' * 50)

    print('分词好的文本：')
    print(c1)  
    print(c2)
    print(c3)
    print('-' * 50)

    # 使用CountVectorizer对中文进行特征值化
    cv = CountVectorizer()

    data = cv.fit_transform([c1, c2, c3])

    print('特征词： \n',cv.get_feature_names_out())  #把处理好后的特征名称打印出来

    print('\n将稀疏矩阵变成密集矩阵：\n',data.toarray())

    return None


# cutword()
chinese_vec()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\19942\AppData\Local\Temp\jieba.cache


jieba分词器产生的结果类型是： <class 'generator'>
--------------------------------------------------


Loading model cost 0.618 seconds.
Prefix dict has been built successfully.


['今天', '很', '残酷', '，', '明天', '更', '残酷', '，', '后天', '很', '美好', '，', '但', '绝对', '大部分', '是', '死', '在', '明天', '晚上', '，', '所以', '每个', '人', '不要', '放弃', '今天', '。']
['我们', '看到', '的', '从', '很', '远', '星系', '来', '的', '光是在', '几百万年', '之前', '发出', '的', '，', '这样', '当', '我们', '看到', '宇宙', '时', '，', '我们', '是', '在', '看', '它', '的', '过去', '。']
['如果', '只用', '一种', '方式', '了解', '某样', '事物', '，', '你', '就', '不会', '真正', '了解', '它', '。', '了解', '事物', '真正', '含义', '的', '秘密', '取决于', '如何', '将', '其', '与', '我们', '所', '了解', '的', '事物', '相', '联系', '。']
--------------------------------------------------
--------------------------------------------------
分词好的文本：
今天 很 残酷 ， 明天 更 残酷 ， 后天 很 美好 ， 但 绝对 大部分 是 死 在 明天 晚上 ， 所以 每个 人 不要 放弃 今天 。
我们 看到 的 从 很 远 星系 来 的 光是在 几百万年 之前 发出 的 ， 这样 当 我们 看到 宇宙 时 ， 我们 是 在 看 它 的 过去 。
如果 只用 一种 方式 了解 某样 事物 ， 你 就 不会 真正 了解 它 。 了解 事物 真正 含义 的 秘密 取决于 如何 将 其 与 我们 所 了解 的 事物 相 联系 。
--------------------------------------------------
特征词： 
 ['一种' '不会' '不要' '之前' '了解' '事物' '今天' '光是在' '几百万年' '发出' '取决于' '只用' '后天' '含义'
 '

# 1.4 tf-idf
用以评估一字词对于一个文件集或一个语料库中的其中一份文件的重要程度<br>
TF-IDF 越高，**说明这个词在这篇文档中很重要，但在其他文档中很少见。而不是所有文档都很常见的（比如'的'）**

In [8]:
# 规范{'l1'，'l2'}，默认='l2'
# 每个输出行都有单位范数，或者：
#
# 'l2'：向量元素的平方和为 1。当应用 l2 范数时，两个向量之间的余弦相似度是它们的点积。
#
# 'l1'：向量元素的绝对值之和为 1。参见preprocessing.normalize。

# smooth_idf布尔值，默认 = True
# 通过在文档频率上加一来平滑 idf 权重，就好像看到一个额外的文档包含集合中的每个术语恰好一次。防止零分裂。
# 比如训练集中有某个词，测试集中没有，就是生僻词，就会造成n(x)分母为零，log(n/n(x)),从而出现零分裂

def tfidf_vec():
    """
    中文特征值化,计算tfidf值
    """
    c1, c2, c3 = cutword()

    print(c1, c2, c3)
    # print(type([c1, c2, c3]))

    # 计算tfidf值
    tf = TfidfVectorizer(smooth_idf=True)

    data = tf.fit_transform([c1, c2, c3])

    print(tf.get_feature_names_out())
    print('-' * 50)
    print(type(data))
    print('-' * 50)
    print(data.toarray())

    return None


tfidf_vec()

jieba分词器产生的结果类型是： <class 'generator'>
--------------------------------------------------
['今天', '很', '残酷', '，', '明天', '更', '残酷', '，', '后天', '很', '美好', '，', '但', '绝对', '大部分', '是', '死', '在', '明天', '晚上', '，', '所以', '每个', '人', '不要', '放弃', '今天', '。']
['我们', '看到', '的', '从', '很', '远', '星系', '来', '的', '光是在', '几百万年', '之前', '发出', '的', '，', '这样', '当', '我们', '看到', '宇宙', '时', '，', '我们', '是', '在', '看', '它', '的', '过去', '。']
['如果', '只用', '一种', '方式', '了解', '某样', '事物', '，', '你', '就', '不会', '真正', '了解', '它', '。', '了解', '事物', '真正', '含义', '的', '秘密', '取决于', '如何', '将', '其', '与', '我们', '所', '了解', '的', '事物', '相', '联系', '。']
--------------------------------------------------
今天 很 残酷 ， 明天 更 残酷 ， 后天 很 美好 ， 但 绝对 大部分 是 死 在 明天 晚上 ， 所以 每个 人 不要 放弃 今天 。 我们 看到 的 从 很 远 星系 来 的 光是在 几百万年 之前 发出 的 ， 这样 当 我们 看到 宇宙 时 ， 我们 是 在 看 它 的 过去 。 如果 只用 一种 方式 了解 某样 事物 ， 你 就 不会 真正 了解 它 。 了解 事物 真正 含义 的 秘密 取决于 如何 将 其 与 我们 所 了解 的 事物 相 联系 。
['一种' '不会' '不要' '之前' '了解' '事物' '今天' '光是在' '几百万年' '发出' '取决于' '只用' '后天' '含义'
 '大部分' '如何' '如果' '宇宙' '我们' '所以

# 2 特征处理，不同的特征拉到到同一个量纲

In [9]:
def mm():
    """
    归一化处理
    """
    # 归一化缺点 容易受极值的影响
    
    # 用MinmaxScaler设置归一化范围，通过feature_range参数设置
    mm = MinMaxScaler(feature_range=(0, 1))

    # fit_transfrom() 训练数据
    train_data = mm.fit_transform([[90, 2, 10, 40], [60, 4, 15, 45], [75, 3, 13, 46]])
    print('归一化训练：\n', train_data)
    print('-' * 50)

    # transform() 测试数据
    test_data = mm.transform([[1, 2, 3, 4], [6, 5, 8, 7]]) #测试集
    print('归一化测试：\n', test_data)

    return None
    #transform和fit_transform不同是，transform用于测试集，而且不会重新找最小值和最大值


mm()

训练：
 [[1.         0.         0.         0.        ]
 [0.         1.         1.         0.83333333]
 [0.5        0.5        0.6        1.        ]]
--------------------------------------------------
测试：
 [[-1.96666667  0.         -1.4        -6.        ]
 [-1.8         1.5        -0.4        -5.5       ]]


In [10]:
(1 - 60) / 30

-1.9666666666666666

In [27]:
def stand():
    """
    标准化,$注意： 不是标准正太分布,只均值为0,方差为1的分布
    $注意： 标准化，是针对每一列进行标准化
    """
    # 引入StandardScaler，用于标准化处理
    std = StandardScaler()

    # fit_transform() 训练数据
    data = std.fit_transform([[1., -1., 3.],
                              [2., 4., 2.],
                              [4., 6., -1.]])

    print('标准化训练：\n', data)
    print('-' * 50)

    print('训练数据的均值：\n', std.mean_)
    print('-' * 50)

    print('训练数据的方差：\n', std.var_)  #方差
    print('训练数据的样本数：\n', std.n_samples_seen_)  # 样本数

    print('-' * 50)
    print('fit_transform() 训练后返回的数据类型：\n', type(data))


data = stand()

标准化训练：
 [[-1.06904497 -1.35873244  0.98058068]
 [-0.26726124  0.33968311  0.39223227]
 [ 1.33630621  1.01904933 -1.37281295]]
--------------------------------------------------
训练数据的均值：
 [2.33333333 3.         1.33333333]
--------------------------------------------------
训练数据的方差：
 [1.55555556 8.66666667 2.88888889]
训练数据的样本数：
 3
--------------------------------------------------
数据类型：
 <class 'numpy.ndarray'>


In [12]:
#如何求方差 （x-mean）**2 / n
(np.square(1 - 2.333) + np.square(2 - 2.333) + np.square(4 - 2.333)) / 3

1.5555556666666668

In [14]:
# 计算每一列，标准化后数据的均值，验证是否接近0
(-1.06904497 + -0.26726124 + 1.33630621) / 3

0.0

##### 验证已经标准化过的数据再次标准化后，均值应该为0，方差应该为1

In [15]:
# 创建一个新的StandardScaler对象用于标准化
std1 = StandardScaler()

# 对已经标准化过的数据进行再次标准化，验证结果
data1 = std1.fit_transform(data)

# 均值
print(std1.mean_)
# 方差
print(std1.var_)

[-1.48029737e-16  7.40148683e-17  7.40148683e-17]
[1. 1. 1.]


#### transform和fit_transform不同是，transform用于测试集，而且不会重新找最小值和最大值,不会重新计算均值方差

# 3 缺失值处理

In [28]:
#下面是填补缺失值，针对删除，可以用pd和np
def im():
    """
    缺失值处理
    SimpleImputer( missing_values=指定谁是缺失值, strategy=指定填补策略)
    """
    # np.nan替换为均值
    # mean, median, most_frequent(众数), constant
    im = SimpleImputer(missing_values=np.nan, strategy='mean')

    # 创建一个包含缺失值的数组
    data = im.fit_transform([[1, 2], [np.nan, 3], [7, 6], [3, 2]])

    # 打印填补后的数据
    print(data)

    return None


im()

[[1.         2.        ]
 [3.66666667 3.        ]
 [7.         6.        ]
 [3.         2.        ]]


# 4 降维（拟合）
降维就是特征数变少<br>
降维可以提高模型训练速度（特征变少）

### 方差阈值

In [29]:
def var():
    """
    特征选择-删除低方差的特征
    """
    #默认只删除方差为0,threshold是方差阈值，删除比这个值小的那些特征， threshold：小数
    var = VarianceThreshold(threshold=0.1)

    # 导入数据训练
    data = var.fit_transform([[0, 2, 0, 3],
                              [0, 1, 4, 3],
                              [0, 1, 1, 3]])

    print('删除了低于10%方差列的训练结果：\n', data)
    print('-' * 50)

    # 获取符合条件的特征列编号
    print('保留下来的列编号 %s' % var.get_support(True))
    return None


var()


删除了低于10%方差列的训练结果：
 [[2 0]
 [1 4]
 [1 1]]
--------------------------------------------------
保留下来的列编号 [1 2]


In [ ]:
# VarianceThreshold用于特征选择，去除方差低于阈值的特征
print("\nVarianceThreshold示例:")

# 创建一个示例数据矩阵
X_variance = np.array([
    [0, 2, 0, 3],
    [0, 1, 4, 3],
    [0, 1, 1, 3]
])
print("原始数据矩阵:\n", X_variance)

# 计算每个特征的方差
feature_variances = np.var(X_variance, axis=0)
print("\n各特征的方差:", feature_variances)

# 使用方差阈值为0.8进行特征选择
# 这将移除方差小于0.8的特征
selector = VarianceThreshold(threshold=0.8)
X_selected = selector.fit_transform(X_variance)

# 显示保留的特征索引
print("\n保留的特征索引:", selector.get_support(indices=True))
print("保留的特征方差:", feature_variances[selector.get_support()])
print("\n特征选择后的数据矩阵:\n", X_selected)

# 尝试不同的阈值
selector_low = VarianceThreshold(threshold=0.1)
X_selected_low = selector_low.fit_transform(X_variance)
print("\n阈值为0.1时保留的特征索引:", selector_low.get_support(indices=True))
print("阈值为0.1时特征选择后的数据矩阵:\n", X_selected_low)


### PCA

In [35]:
def pca():
    """
    主成分分析进行特征降维
    """
    # n_ components:小数 0~1 业界选择 90~95% 。建议 90%

    # 当n_components的值为0到1之间的浮点数时，表示我们希望保留的主成分解释的方差比例。方差比例是指 得到输出的每一列的方差值和除以原有数据方差之和。
    # n_components如果是整数   代表要把列（特征）降低到多少列

    # 原始数据方差
    original_value = np.array([[2, 8, 4, 5],
                               [6, 3, 0, 8],
                               [5, 4, 9, 1]])

    print('最初数据每一列的方差和：\n', np.var(original_value, axis=0).sum())  
    print('-' * 50)

    # 创建PCA对象，当前几个主成分的方差贡献率和=90%，保留这几个主成分，而
    # 小数的优点：不需要手动指定降低到多少列
    pca = PCA(n_components=0.9)

    # 用PCA 训练数据
    data = pca.fit_transform(original_value)

    print('PCA训练后的数据：\n', data)
    print('-' * 50)
    print('PCA训练后的数据类型：\n', type(data))
    print('-' * 50)

    #计算data的方差和
    print('PCA训练后的数据每一列的方差和：\n', np.var(data, axis=0))
    print('-' * 50)

    # 计算data的方差占总方差的比例
  
    # 第一个主成分解释了75%的方差
    # 75%不够90%，所以需要第二个主成分，二个主成分解释了25%的方差，
    print('PCA训练后的数据每一列的方差占总方差的比例：\n', pca.explained_variance_ratio_)
    print('-' * 50)
    
    # 计算data的方差占总方差的比例,加起来是1
    print(pca.explained_variance_ratio_.sum())

    return None


pca()

最初数据每一列的方差和：
 29.333333333333336
--------------------------------------------------
PCA训练后的数据：
 [[-1.28620952e-15  3.82970843e+00]
 [-5.74456265e+00 -1.91485422e+00]
 [ 5.74456265e+00 -1.91485422e+00]]
--------------------------------------------------
PCA训练后的数据类型：
 <class 'numpy.ndarray'>
--------------------------------------------------
PCA训练后的数据每一列的方差和：
 [22.          7.33333333]
--------------------------------------------------
PCA训练后的数据每一列的方差占总方差的比例：
 [0.75 0.25]
--------------------------------------------------
1.0


In [23]:
# PCA训练后方差应该非常接近原data的方差
29.333333333333332 / 29.333333333333336

0.9999999999999999